### Test Prophet model over 20 randomly-selected Community Areas

In [ ]:
import zipfile
import pandas as pd
from sklearn.preprocessing import StandardScaler
from prophet import Prophet
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
import logging

In [ ]:
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In [ ]:
# load data for testing (20 randomly-selected community areas x 4 time resolutions)
ts_fn = '/content/drive/MyDrive/Data/ts_files.zip'
zz = zipfile.ZipFile(ts_fn, 'r')
zz.extractall()
zz.close()

In [ ]:
dn = '/content/file upload'
fns = os.listdir(dn)
len(fns)

80

In [ ]:
# get rmse, mae, and mean count for each test file
rmse_list, mae_list, y_test_m_list, fn_list = ([] for xx in range(4))

for fn in tqdm(fns):
  df = pd.read_csv(os.path.join(dn, fn), index_col=0)
  t_res = fn.split('.')[0].split('_')[-1].upper()

  df1 = df.copy()
  spatial_region = df1['Community Area'].unique()[0]
  df1['ds'] = pd.to_datetime(df1['DateTime'])
  df1.rename(columns={'BullsGame':'bulls', 'CubsGame':'cubs', 'SoxGame':'sox', 'BearsGame':'bears', 
                    'C':'temp', 'm/s':'wind', 'PRCP':'rain', 'SNOW':'snow', 'Count':'y'}, inplace=True)


  holidays = pd.DataFrame({'holiday': 'holiday',
                          'ds': df1[df1['Holiday']==1]['ds'],
                          'lower_window': 0, 'upper_window': 1,})

  cols2drop = ['Date', 'HourGroup', 'DateTime', 'Year', 'Month', 'Day', 'Weekday', 'Holiday', 'Community Area']
  df1.drop(cols2drop, axis=1, inplace=True)

  assert df1.isna().sum().sum()==0

  test_length = int(df1.shape[0]*0.2)
  train = df1.iloc[:-test_length].copy()
  test = df1.iloc[df1.shape[0]-test_length:].copy()
  # print(len(train), len(test))

  cols2scale = ['temp', 'wind', 'rain', 'snow']
  scaler = StandardScaler().fit(train[cols2scale])
  train[cols2scale] = scaler.transform(train[cols2scale])
  test[cols2scale] = scaler.transform(test[cols2scale])
  # train.shape, test.shape

  m = Prophet(holidays=holidays)
  # m = Prophet()
  extra_regressors = ['bulls', 'cubs', 'sox', 'bears', 'temp', 'wind', 'rain', 'snow']
  for regressor in extra_regressors:
    m.add_regressor(regressor)
  m.fit(train)

  future = m.make_future_dataframe(periods=test_length, freq=t_res)
  tmp = pd.concat([train, test]).drop('y', axis=1)
  future = future.merge(tmp, how='left', on='ds')
  # future.tail()

  forecast = m.predict(future)
  results = test[['ds', 'y']].merge(forecast[['ds', 'yhat']])
  
  rmse = mean_squared_error(y_true=results['y'], y_pred=results['yhat'], squared=False)
  mae = mean_absolute_error(y_true=results['y'], y_pred=results['yhat'])
  rmse_list.append(rmse)
  mae_list.append(mae)
  y_test_m_list.append(results['y'].mean())
  fn_list.append(fn.split('/')[-1])

output_df = pd.DataFrame({'fn':fn_list, 'rmse':rmse_list, 'mae':mae_list, 'y_test_m':y_test_m_list,})

100%|██████████| 80/80 [1:03:50<00:00, 47.88s/it]


In [ ]:
output_df

,fn,rmse,mae,y_test_m
0,crimes_agg_ext_beat14_6h.csv,1.320846,1.031596,1.365145
1,crimes_agg_ext_beat4_4h.csv,0.986944,0.785995,0.811767
2,crimes_agg_ext_beat47_8h.csv,0.581846,0.449676,0.307515
3,crimes_agg_ext_beat14_8h.csv,1.549964,1.220260,1.820194
4,crimes_agg_ext_beat25_8h.csv,6.078193,4.822598,11.592669
...,...,...,...,...
75,crimes_agg_ext_beat20_2h.csv,0.552146,0.406487,0.280141
76,crimes_agg_ext_beat47_2h.csv,0.282953,0.147444,0.076879
77,crimes_agg_ext_beat47_4h.csv,0.403717,0.267010,0.153757
78,crimes_agg_ext_beat31_2h.csv,0.771749,0.594335,0.518154


In [ ]:
# output_df.to_csv('/content/drive/MyDrive/prophet_results.csv')

In [ ]:
# results summary
# output_df = pd.read_csv('/content/drive/MyDrive/prophet_results.csv', index_col=0)
output_df['hour'] = output_df['fn'].apply(lambda x: x.split('.')[0].split('_')[-1][:-1])
output_df['comm_area'] = output_df['fn'].apply(lambda x: x.split('_')[-2].split('beat')[-1])
output_df['norm_rmse'] = output_df['rmse']/output_df['y_test_m']
output_df['norm_mae'] = output_df['mae']/output_df['y_test_m']
output_dfg = output_df.groupby(['hour']).mean()
# output_dfg.to_csv('/content/drive/MyDrive/prophet_results_summary2.csv')
output_dfg

,rmse,mae,y_test_m,norm_rmse,norm_mae
hour,,,,,
2,0.947620,0.737365,0.809793,1.634770,1.183084
4,1.457963,1.150556,1.619553,1.216080,0.927936
6,1.886737,1.490912,2.428423,1.027853,0.801074
8,2.287711,1.813585,3.239107,0.916870,0.718537
